In [ ]:
import anndata
import os
import requests

def read_link(link, save_path):
    if not os.path.exists(save_path):
        response = requests.get(link)
        with open(save_path, "wb") as f:
            f.write(response.content)

    return anndata.read_h5ad(save_path)

MOBSC_sce = read_link("https://go.wisc.edu/yeo402", "data/MOBSC_sce.h5ad")
MOBSP_sce = read_link("https://go.wisc.edu/n7bcr6", "data/MOBSP_sce.h5ad")

exper = {
    "sc": read_link("https://go.wisc.edu/yeo402", "data/MOBSC_sce.h5ad"),
    "sp": read_link("https://go.wisc.edu/n7bcr6", "data/MOBSP_sce.h5ad")
    }

We haven't implemented differential variance in the current copula estimator! This is a simple extension -- let's make sure to add it.

In [ ]:
from scdesigner.simulators import NegBinCopulaSimulator

formulas = {"sc": "~ cellType", "sp": "~ bs(spatial1, df=40) + bs(spatial2, df=40)"}
sims = {k: NegBinCopulaSimulator(epochs=5) for k in exper.keys()}
{v.fit(exper[k], formulas[k]) for k, v in sims.items()}

In [ ]:
sims["sc"]

We need also a function to get the mean matrices. This is like "predict" in R.

In [ ]:
means = {k: v.predict(exper[k].obs)["mean"] for k, v in sims.items()}

In [ ]:
means

In [ ]:
import tangram as tg
from copy import deepcopy

mean_adata = {k: deepcopy(v) for k, v in exper.items()}
for k in exper.keys():
    mean_adata[k].X = means[k]

tg.pp_adatas(exper["sc"], exper["sp"])
adata_map = tg.map_cells_to_space(exper["sc"], exper["sp"], num_epochs=1)

In [ ]:
import tangram.plot_utils as pl

pl.plot_cell_annotation(adata_map, exper["sp"], x="spatial1", y="spatial2", ncols=4)

In [ ]:
tg.project_cell_annotations(adata_map, exper["sp"])
mixtures = exper["sp"].obsm["tangram_ct_pred"]

In [ ]:
import numpy as np

exper["sp_sim"] = exper["sp"].copy()
exper["sp_sim"].X = np.zeros_like(exper["sp"].X)
cell_types = list(exper["sc"].obs["cell_type"].dtype.categories)
n_cells = len(exper["sp"])
cells_per_spot = 50, 5

In [ ]:
for n in range(n_cells):
    for k in cell_types:
        # get a random subset of cells of this type
        candidate = np.where(exper["sc"].obs["cell_type"] == k)[0]
        ix = np.random.choice(candidate, size=cells_per_spot[0])

        # simulate according to the deconvolution result
        depths = exper["sc"].X[ix, :].sum(axis=0)
        exper["sp_sim"].X[[n], :] += np.ceil(depths * mixtures[k][n] / cells_per_spot[1])

In [ ]:
exper["sp_sim"]

In [ ]:
import squidpy as sq

sq.pl.spatial_scatter(exper["sp"], colors=[""])

In [ ]:
signatures = np.zeros((len(cell_types), exper["sc"].n_vars))
for k, cell_type in enumerate(cell_types):
    ix = exper["sc"].obs["cellType"] == cell_type
    signatures[k, :] = exper["sc"][ix].X.mean(axis=0)[0]

sc_mixture = mixtures @ signatures

In [ ]:
sc_mixture